# buffering/缓冲区

In [1]:
# buffering是一个内存空间，一个FIFO(first in first out)队列，缓冲区满了，数据会flush到磁盘。

# -1 表示使用缺省大小的buffer。如果是二进制，使用io.DEFAULT_BUFFER_SIZE，默认是4096或8192。
# 如果是文本模式、终端设备，则是行缓存方式，如果不是，则使用二进制模式的策略。
# 0 只在二进制模式使用，表示关buffer
# 1 只在文本模式使用，表示使用行缓冲。不超出缓冲区极限时，即见到换行符就flush
# 大于1 用于指定buffer的大小

# flush() 将缓冲区数据写入磁盘
# close() 关闭前会调用flush()

In [ ]:
f = open("test", "wb+", 0)  # 二进制模式下，关闭缓冲区，立即看到写入结果
# f = open("test", "w+", 0)  # 字符流模式下，不能关闭缓冲区
f = open("test", "wb+", 4)  # 只开辟了4个字节的缓冲区，如果一次的写入超过4个字节，会直接flush所有写入内容

# 行读取

In [3]:
f.readline(size=-1)  # 默认读取一行，size设定一次读取的字符数
f.readline(1)  # 按指定的size，逐个字符读取
for line in f.readline(hint=-1):  # 读取所有行，hint无视掉
    print(line.strip())
    
for line in f:  # 推荐
    print(line.strip())  # 去除无效的空行

# 其他

In [ ]:
f.seekable()  # 是否可seek
f.readable()  # 是否可读
f.writable()  # 是否可写

# 上下文管理

In [ ]:
# 上下文管理：
#   使用with ... as 关键字
#   上下文管理的语句块并不会开启新的作用域
#   with语句块执行完后，会自动关闭文件对象

In [6]:
with open("text", "w+") as f:
    f.write(str(1/0))

ZeroDivisionError: division by zero

In [7]:
print(f.closed)  # with语法，会在出现异常时，自动关闭f

True


# 习题

In [8]:
# 统计一片文章中的高频词语

In [14]:
chars = set(r""",."'[](){}-+/\*&#$@""")  # 特殊字符集合

def makekey(x):
    key = x.lower()
    res = []
    for c in key:
        if c in chars:  # 处理方法优于replace
            res.append("")
        else:
            res.append(c)
    return "".join(res).split()  # 多个元素的集合

out = {}

with open("log.txt", encoding="utf-8") as f:
    for line in f:
        words = line.split()
        for wordlist in map(makekey, words):
            for word in wordlist:
                out[word] = out.get(word, 0) + 1
                
print(sorted(out.items(), key=lambda item:item[1], reverse=True))

[('safasdf', 1), ('path', 1), ('asdfa', 1)]


# StringIO、BytesIO模块

In [ ]:
# StringIO好处：
#     磁盘的操作比内存要慢得多，内存足够时，优化的思路是少落地，减少磁盘IO的过程，可大大提高程序的运行效率
# BytesIO：
#     在内存中开辟一个二进制模式的buffer，可以向文件对象一样操作它。

In [15]:
from io import StringIO, BytesIO

sio = StringIO()  # 类文件对象
print(sio.seekable(), sio.readable(), sio.writable())

True True True


In [17]:
with StringIO() as sio:  # 如果类中存在__Enter__、__Exit__即可调用with ... as
    print(sio.write("Hello Python"))
    print(sio.seek(0))
    print(sio.read())

12
0
Hello Python


In [18]:
with BytesIO() as sio:  # 如果类中存在__Enter__、__Exit__即可调用with ... as
    print(sio.write(b"Hello Python"))
    print(sio.seek(0))
    print(sio.read())

12
0
b'Hello Python'


In [ ]:
# 类文件对象：可以像文件一样进行操作，socket、输入输出(stdin、stdout)都是类文件对象

# Path模块

In [19]:
from os import path

p = path.join("/etc", "sysconfig", "network")  # 当前操作系统的目录分隔符
p  # 相对路径

'/etc\\sysconfig\\network'

In [22]:
path.abspath(p)  # 绝对路径

'C:\\etc\\sysconfig\\network'

In [23]:
path.abspath(".")

'C:\\Users\\Administrator\\AAAPython3'

In [24]:
path.split(path.abspath(p))

('C:\\etc\\sysconfig', 'network')

In [25]:
path.dirname(path.abspath(p))  # 上层路径目录

'C:\\etc\\sysconfig'

In [26]:
path.dirname(path.dirname(path.abspath(p)))

'C:\\etc'

In [28]:
print(path.dirname(p), path.basename(p))  # 拆分
print(path.splitdrive(path.abspath(p)))  # 根据磁盘分割

/etc\sysconfig network
('C:', '\\etc\\sysconfig\\network')


In [30]:
# 以上os是老版本的
# 下面是3.4以后推荐的

In [32]:
from pathlib import Path

p = Path()
print(p, p.absolute())  # 当前路径的绝对路径

. C:\Users\Administrator\AAAPython3


In [37]:
p0 = p/"a/b/c/d"  # 推荐，方便简洁
p1 = p/"a"/"b"/"c/d"
p2 = Path().joinpath("a", "b", "c/d")
p0, p1, p2

(WindowsPath('a/b/c/d'), WindowsPath('a/b/c/d'), WindowsPath('a/b/c/d'))

In [39]:
print(p1.absolute().parts)  # 分解
print(p1.absolute().parent.parent)  # 取父目录
for pp in p1.absolute().parents:
    print(pp)

('C:\\', 'Users', 'Administrator', 'AAAPython3', 'a', 'b', 'c', 'd')
C:\Users\Administrator\AAAPython3\a\b
C:\Users\Administrator\AAAPython3\a\b\c
C:\Users\Administrator\AAAPython3\a\b
C:\Users\Administrator\AAAPython3\a
C:\Users\Administrator\AAAPython3
C:\Users\Administrator
C:\Users
C:\


In [ ]:
p2.with_name("xx.py")  # 替换文件名称
p2.with_suffix("xx.py")  # 替换文件的后缀

# shutil模块

In [43]:
import shutil

with open("d:/temp/test.txt", "r+") as f1:
    f1.write("xiwowangyi\nyangliuyiyi")
    f1.flush
    f1.seek(0)  # !
    with open("d:/temp/test1.txt", "w") as f2:
        shutil.copyfileobj(f1, f2)

In [ ]:
shutil.copy(src, dst[, ])
shutil.copy2(src, dst[, ])
shutil.copyfile(src, dst[, ])
shutil.copymode(src, dst[, ])
shutil.copystat(src, dst[, ])
shutil.copytree(src, dst[, ])

# shutil.remtree("d:/temp/test.txt")  # rm -rf /

# ini

In [ ]:
# 配置文件 
# 中括号里面的部分叫section，每一个section中，都是key=value的形式，key被称为option
[DEFAULT]  # 缺省区，这个必须大写，特殊
